This notebook mainly focuses on the implementation of a Deep Learning model.

An Artificial Neural network will be created to test on the given dataset.

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')

In [2]:
raw_data = pd.read_csv('propulsion.csv')

data = raw_data.drop('Unnamed: 0', axis = 1)

data.head()

,Lever position (lp) [ ],Ship speed (v) [knots],Gas Turbine shaft torque (GTT) [kN m],Gas Turbine rate of revolutions (GTn) [rpm],Gas Generator rate of revolutions (GGn) [rpm],Starboard Propeller Torque (Ts) [kN],Port Propeller Torque (Tp) [kN],HP Turbine exit temperature (T48) [C],GT Compressor inlet air temperature (T1) [C],GT Compressor outlet air temperature (T2) [C],HP Turbine exit pressure (P48) [bar],GT Compressor inlet air pressure (P1) [bar],GT Compressor outlet air pressure (P2) [bar],Gas Turbine exhaust gas pressure (Pexh) [bar],Turbine Injecton Control (TIC) [%],Fuel flow (mf) [kg/s],GT Compressor decay state coefficient.,GT Turbine decay state coefficient.
0,9.300,27.0,72762.205,3560.393,9753.812,644.806,644.806,1086.583,288.0,780.304,4.523,0.998,22.879,1.050,90.435,1.790,0.973,0.978
1,6.175,18.0,29760.552,2306.825,8780.012,246.011,246.011,776.921,288.0,665.511,2.518,0.998,13.438,1.030,34.596,0.685,0.995,0.975
2,3.144,9.0,8375.774,1386.739,7051.621,60.318,60.318,589.764,288.0,580.587,1.392,0.998,7.566,1.020,12.477,0.247,0.980,0.978
3,3.144,9.0,8377.589,1386.748,7098.469,60.339,60.339,570.651,288.0,576.565,1.390,0.998,7.409,1.021,11.678,0.231,0.984,1.000
4,6.175,18.0,29761.091,2306.825,8782.024,246.021,246.021,769.855,288.0,663.682,2.518,0.998,13.374,1.031,34.154,0.676,0.998,0.980


#####   

## Splitting training, validation and test datasets

In [3]:
x = data.drop(["GT Compressor decay state coefficient.", "GT Turbine decay state coefficient."], axis = 1)
y = data['GT Compressor decay state coefficient.']

In [4]:
from sklearn.model_selection import train_test_split

#Training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 42)

#Training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state = 42)

print("Training set shape: ", x_train.shape, y_train.shape)
print("Validation set shape: ", x_val.shape, y_val.shape)
print("Test set shape: ", x_test.shape, y_test.shape)

Training set shape:  (8100, 16) (8100,)
Validation set shape:  (900, 16) (900,)
Test set shape:  (1000, 16) (1000,)


## Scaling the data

In [5]:
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler() 
  
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test) 

###   

## Neural Network model Creation 

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import Adam

In [7]:
model = Sequential() #First creating a sequential object

layer_size = x.shape[1] #Input layer size

#Input Layer
model.add(Dense(128, activation='relu', input_dim = layer_size, kernel_initializer= 'normal'))

#Hidden Layers

#1st hidden layer
model.add(Dense(256, kernel_initializer ='normal', activation='relu'))
model.add(Dropout(0.5)) 

#2nd hidden layer
model.add(Dense(512, kernel_initializer = 'normal', activation='relu'))
model.add(Dropout(0.5))

#3rd hidden layer
model.add(Dense(256, kernel_initializer = 'normal', activation='relu'))
model.add(Dropout(0.5))

#4th hidden layer
model.add(Dense(128, kernel_initializer = 'normal', activation='relu'))
model.add(Dropout(0.5))

#Output layer
model.add(Dense(1, kernel_initializer = 'normal', activation='linear'))

lr = 0.03
#Optimizer parameter tuning
adam = Adam(learning_rate = lr)

#Model compilation
model.compile(loss='mean_squared_error', optimizer = adam, metrics=['mean_squared_error'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               2176      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

## Training the Neural network model

In [8]:
#Early stopping prevents overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 3, restore_best_weights= True)

#Training the model
model.fit(
    
    x_train_scaled, 
    y_train,
    epochs = 100, #Number of iterations
    batch_size = 32,
    callbacks = [early_stopping],
    validation_data = (x_val_scaled, y_val),
    verbose = 2,

)

Train on 8100 samples, validate on 900 samples
Epoch 1/100
8100/8100 - 2s - loss: 21.7692 - mean_squared_error: 21.7692 - val_loss: 0.0771 - val_mean_squared_error: 0.0771
Epoch 2/100
8100/8100 - 1s - loss: 0.0162 - mean_squared_error: 0.0162 - val_loss: 4.7618e-04 - val_mean_squared_error: 4.7618e-04
Epoch 3/100
8100/8100 - 1s - loss: 2.5309e-04 - mean_squared_error: 2.5309e-04 - val_loss: 2.2161e-04 - val_mean_squared_error: 2.2161e-04
Epoch 4/100
8100/8100 - 1s - loss: 2.1639e-04 - mean_squared_error: 2.1639e-04 - val_loss: 2.2165e-04 - val_mean_squared_error: 2.2165e-04
Epoch 5/100
8100/8100 - 1s - loss: 2.1636e-04 - mean_squared_error: 2.1636e-04 - val_loss: 2.2166e-04 - val_mean_squared_error: 2.2166e-04
Epoch 6/100
8100/8100 - 1s - loss: 2.1638e-04 - mean_squared_error: 2.1638e-04 - val_loss: 2.2178e-04 - val_mean_squared_error: 2.2178e-04


## Testing on test set

In [9]:
y_test_pred = model.predict(x_test_scaled) #Testing on test set

rmse = np.sqrt(mean_squared_error(y_test, y_test_pred)) #Evaluating model rmse score by comparing actual and predicted results

print("RMSE score: ", rmse)

RMSE score:  0.014810978470254454


**The performance of this Neural network model could'nt quite catch up with the CatBoost model.**

The RMSE score is lower than that of CatBoost model.

**Hence we can consider that CatBoost still performes exceptionally well.**